In [64]:
# === Cell A: Imports & robust project paths (REPLACE this whole cell) ===
from pathlib import Path
import re
import numpy as np
import pandas as pd

def find_root_with_csv(start: Path,
                       subpath: str = "data/raw",
                       prefer_pat: str = r"PecanStreet.*1Min.*\.(?i:csv)") -> tuple[Path, Path, list[Path]]:
    """
    Walk upward from 'start' and return the first directory whose 'subpath'
    contains at least one CSV. Prefer files matching 'prefer_pat'. Returns:
      (project_root, data_raw_path, candidates)
    """
    cur = start.resolve()
    prefer_re = re.compile(prefer_pat)
    while True:
        candidate_raw = cur / subpath
        if candidate_raw.exists() and candidate_raw.is_dir():
            # gather CSVs (case-insensitive)
            csvs = list(candidate_raw.glob("*.csv")) + list(candidate_raw.glob("*.CSV"))
            # prefer PecanStreet-style names if present
            preferred = [p for p in csvs if prefer_re.search(p.name)]
            picks = preferred if preferred else csvs
            if picks:
                return cur, candidate_raw, picks
        if cur.parent == cur:
            break
        cur = cur.parent
    raise FileNotFoundError(
        f"Could not find a '{subpath}' folder with any CSVs starting from {start} upward."
    )

# Find the project root whose data/raw actually has CSVs
PROJECT_ROOT, DATA_RAW, RAW_CANDIDATES = find_root_with_csv(Path.cwd(), "data/raw")

# Pick the first preferred CSV
RAW_FILE = RAW_CANDIDATES[0]

# Output dirs
DATA_PROCESSED = PROJECT_ROOT / "data" / "processed"
REPORTS_DIR    = PROJECT_ROOT / "reports"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Homes we care about
HOMES = [f"h{i}" for i in range(1, 6)]  # h1..h5

# Debug prints so you can verify
print("Project root:", PROJECT_ROOT)
print("Using raw folder:", DATA_RAW)
print("Found CSVs:", [p.name for p in RAW_CANDIDATES])
print("Using raw file:", RAW_FILE)


Project root: E:\VPP
Using raw folder: E:\VPP\data\raw
Found CSVs: ['PecanStreet_10_Homes_1Min_Data.csv', 'PecanStreet_10_Homes_1Min_Data.csv']
Using raw file: E:\VPP\data\raw\PecanStreet_10_Homes_1Min_Data.csv


In [65]:
# === Cell B: Load raw ===
print("Loading:", RAW_FILE)
raw = pd.read_csv(RAW_FILE)

# Find datetime column
dt_col = None
for c in raw.columns:
    if re.search(r"time|date", c, re.I):
        dt_col = c; break
assert dt_col is not None, "Could not find a datetime column (look for 'time'/'date' in headers)."

raw[dt_col] = pd.to_datetime(raw[dt_col], errors="raise", utc=True)
raw = raw.set_index(dt_col).sort_index()
print("Raw shape:", raw.shape)
raw.head(3)


Loading: E:\VPP\data\raw\PecanStreet_10_Homes_1Min_Data.csv
Raw shape: (993830, 4)


,Home ID,Circuit,Measure,Value
Datetime (UTC),,,,
2018-08-23 00:00:00+00:00,1,HVAC Condenser,Real Power,3.802
2018-08-23 00:00:00+00:00,10,Solar,Real Power,0.074
2018-08-23 00:00:00+00:00,7,Main Panel,Apparent Power,4.662


In [66]:
# === Cell C: Select homes & pivot to wide (replace your old Cell C) ===
DT = "Datetime (UTC)"
HOME = "Home ID"
CIR = "Circuit"
MEAS = "Measure"
VAL = "Value"

df = raw.copy()
# keep real power only
df = df[df[MEAS].str.contains("real", case=False, na=False)]

# find homes that have both Solar and Main Panel data
has_solar = set(df[df[CIR].str.contains("solar", case=False, na=False)][HOME].unique())
has_main  = set(df[df[CIR].str.contains("main panel", case=False, na=False)][HOME].unique())
candidates = sorted(has_solar & has_main)
assert candidates, "No homes have both Solar and Main Panel Real Power."

# pick the first 5 consistently
ids = candidates[:5]
assert len(ids) >= 5, f"Need 5 homes; only found {len(ids)} with both signals."

# map numeric IDs -> h1..h5 (deterministic order)
id2h = {hid: f"h{i+1}" for i, hid in enumerate(ids)}
HOMES = [id2h[i] for i in ids]
print("Selected homes (ID -> label):", id2h)

# helper to pivot a given circuit to wide
def pivot_circuit(df_long: pd.DataFrame, circuit_name: str) -> pd.DataFrame:
    sub = df_long[df_long[CIR].str.contains(circuit_name, case=False, na=False) & df_long[HOME].isin(ids)]
    wide = (sub
            .pivot_table(index=DT, columns=HOME, values=VAL, aggfunc="mean")
            .sort_index())
    # rename columns to h1..h5
    wide = wide.rename(columns=id2h)
    # ensure Datetime index tz-aware in UTC
    wide.index = pd.to_datetime(wide.index, utc=True)
    return wide

pv_1min   = pivot_circuit(df, "solar")
load_1min = pivot_circuit(df, "main panel")

print("pv_1min shape:", pv_1min.shape)
print("load_1min shape:", load_1min.shape)
display(pv_1min.head(3))
display(load_1min.head(3))


Selected homes (ID -> label): {np.int64(1): 'h1', np.int64(2): 'h2', np.int64(3): 'h3', np.int64(4): 'h4', np.int64(5): 'h5'}
pv_1min shape: (4321, 5)
load_1min shape: (4321, 5)


Home ID,h1,h2,h3,h4,h5
Datetime (UTC),,,,,
2018-08-23 00:00:00+00:00,1.556,0.932,0.220,1.441,0.732
2018-08-23 00:01:00+00:00,1.536,0.917,0.214,1.420,0.718
2018-08-23 00:02:00+00:00,1.512,0.901,0.207,1.390,0.704


Home ID,h1,h2,h3,h4,h5
Datetime (UTC),,,,,
2018-08-23 00:00:00+00:00,3.339,1.320,3.036,1.653,5.606
2018-08-23 00:01:00+00:00,3.358,1.335,3.143,1.672,5.633
2018-08-23 00:02:00+00:00,3.365,1.332,3.149,1.694,5.627


In [67]:
# === Cell D: Normalize to kW ===
def to_kW(wide: pd.DataFrame) -> pd.DataFrame:
    vals = wide[wide > 0].stack()
    if not vals.empty and vals.median() > 100:   # looks like Watts
        return wide / 1000.0
    return wide

pv_1min_kW   = to_kW(pv_1min).astype(float)
load_1min_kW = to_kW(load_1min).astype(float)

print("pv_1min_kW sample:"); display(pv_1min_kW.head(3))
print("load_1min_kW sample:"); display(load_1min_kW.head(3))


pv_1min_kW sample:


Home ID,h1,h2,h3,h4,h5
Datetime (UTC),,,,,
2018-08-23 00:00:00+00:00,1.556,0.932,0.220,1.441,0.732
2018-08-23 00:01:00+00:00,1.536,0.917,0.214,1.420,0.718
2018-08-23 00:02:00+00:00,1.512,0.901,0.207,1.390,0.704


load_1min_kW sample:


Home ID,h1,h2,h3,h4,h5
Datetime (UTC),,,,,
2018-08-23 00:00:00+00:00,3.339,1.320,3.036,1.653,5.606
2018-08-23 00:01:00+00:00,3.358,1.335,3.143,1.672,5.633
2018-08-23 00:02:00+00:00,3.365,1.332,3.149,1.694,5.627


In [68]:
# === Cell E: Resample to 10-min & compute surplus ===
def resample_10min_mean(df):
    return (df
            .resample("10T", origin="start")
            .mean()
            .sort_index())

pv_10   = resample_10min_mean(pv_1min_kW)
load_10 = resample_10min_mean(load_1min_kW)

pv_10, load_10 = pv_10.align(load_10, join="inner")
surplus_10 = (pv_10 - load_10).astype(float)

print(pv_10.shape, load_10.shape, surplus_10.shape)
display(surplus_10.head(3))


(433, 5) (433, 5) (433, 5)


C:\Users\Jasse\AppData\Local\Temp\ipykernel_27200\706073555.py:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  .resample("10T", origin="start")


Home ID,h1,h2,h3,h4,h5
Datetime (UTC),,,,,
2018-08-23 00:00:00+00:00,-2.0400,-3.1263,-2.9052,-0.3759,-4.9699
2018-08-23 00:10:00+00:00,-2.5241,-6.2338,-2.9538,-0.7642,-7.6847
2018-08-23 00:20:00+00:00,-3.5272,-2.0854,-3.1524,-1.7187,-5.9720


In [69]:
# === Cell F: Save processed outputs ===
PV_10MIN_CSV      = DATA_PROCESSED / "pv_kW_10min.csv"
LOAD_10MIN_CSV    = DATA_PROCESSED / "load_kW_10min.csv"
SURPLUS_10MIN_CSV = DATA_PROCESSED / "surplus_kW_10min.csv"

pv_10.to_csv(PV_10MIN_CSV, date_format="%Y-%m-%d %H:%M:%S%z")
load_10.to_csv(LOAD_10MIN_CSV, date_format="%Y-%m-%d %H:%M:%S%z")
surplus_10.to_csv(SURPLUS_10MIN_CSV, date_format="%Y-%m-%d %H:%M:%S%z")

print("Wrote:", PV_10MIN_CSV, LOAD_10MIN_CSV, SURPLUS_10MIN_CSV, sep="\n - ")


Wrote:
 - E:\VPP\data\processed\pv_kW_10min.csv
 - E:\VPP\data\processed\load_kW_10min.csv
 - E:\VPP\data\processed\surplus_kW_10min.csv


In [70]:
# === Cell G: Seller flags & windows ===
SELLER_FLAGS_CSV = DATA_PROCESSED / "seller_flags_10min.csv"
seller_flags = surplus_10.gt(0.0)
seller_flags.to_csv(SELLER_FLAGS_CSV, date_format="%Y-%m-%d %H:%M:%S%z")

from itertools import groupby

def contiguous_windows(mask_series: pd.Series):
    wins, on, start, prev = [], False, None, None
    for ts, ok in mask_series.items():
        if ok and not on: on, start = True, ts
        elif not ok and on: wins.append((start, prev)); on = False
        prev = ts
    if on: wins.append((start, prev))
    return wins

WINDOWS_CSV = (PROJECT_ROOT / "reports" / "seller_windows.csv")
rows = []
for h in seller_flags.columns:
    for a, b in contiguous_windows(seller_flags[h]):
        rows.append({"home": h, "start": a, "end": b})
pd.DataFrame(rows).to_csv(WINDOWS_CSV, index=False)
print("Wrote:\n -", SELLER_FLAGS_CSV, "\n -", WINDOWS_CSV)


Wrote:
 - E:\VPP\data\processed\seller_flags_10min.csv 
 - E:\VPP\reports\seller_windows.csv


In [71]:
# === Cell H: Meta & QA ===
META_TXT = PROJECT_ROOT / "prepare_meta.txt"
def summarize(df, name):
    return f"{name}: shape={df.shape}, start={df.index.min()}, end={df.index.max()}, cols={list(df.columns)}"
meta = "\n".join([
    summarize(pv_10, "PV_10min(kW)"),
    summarize(load_10, "LOAD_10min(kW)"),
    summarize(surplus_10, "SURPLUS_10min(kW=PV-LOAD)")
])
META_TXT.write_text(meta, encoding="utf-8")
print(meta)


PV_10min(kW): shape=(433, 5), start=2018-08-23 00:00:00+00:00, end=2018-08-26 00:00:00+00:00, cols=['h1', 'h2', 'h3', 'h4', 'h5']
LOAD_10min(kW): shape=(433, 5), start=2018-08-23 00:00:00+00:00, end=2018-08-26 00:00:00+00:00, cols=['h1', 'h2', 'h3', 'h4', 'h5']
SURPLUS_10min(kW=PV-LOAD): shape=(433, 5), start=2018-08-23 00:00:00+00:00, end=2018-08-26 00:00:00+00:00, cols=['h1', 'h2', 'h3', 'h4', 'h5']
